## Imports

In [ ]:
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parent
sys.path.append(str(PROJECT_DIR))

# basics
import pandas as pd

# utils
import os
from pyod import models
from ppm.nodes.plots import (
    plot_outliers_histogram
)
from ppm.nodes.outliers_detection import (
    detect_outliers
)

## Parameters

In [ ]:
target = "price"

path_root = os.path.join(
    "..",
    "data"
)
path_primary = os.path.join(
    path_root, "03_primary"
)
path_model = os.path.join(
    path_root, "04_model"
)

file_path_input_data = os.path.join(
    path_primary, "data_input.csv"
)
file_path_not_outliers = os.path.join(
    path_primary, "data_not_outliers.csv"
)

## Read dataset

In [ ]:
data_input = pd.read_csv(
    file_path_input_data,
    index_col = 0
)

## Outlier vizualiation

In [ ]:
lower_limit, upper_limit = detect_outliers(data_input[target].values, p = 25)

In [ ]:
plot_outliers = plot_outliers_histogram(data_input, target, lower_limit, upper_limit)

In [ ]:
data_not_outliers = data_input[
    (data_input[target] >= lower_limit)&(data_input[target] <= upper_limit)
].reset_index(drop = True)

## Not outliers

In [ ]:
plot_not_outliers = plot_outliers_histogram(data_not_outliers, target, lower_limit, upper_limit)

## Save not outliers

In [ ]:
data_not_outliers.to_csv(file_path_not_outliers)